In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install jovian opendatasets --upgrade --quiet
import opendatasets as od

     |████████████████████████████████| 68 kB 3.9 MB/s 


In [2]:
dataset_url = 'https://www.kaggle.com/omeret/not-safe-for-work'

In [3]:
!pip install jovian opendatasets --upgrade --quiet
import opendatasets as od
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: Your Kaggle Key: Downloading not-safe-for-work.zip to ./not-safe-for-work


100%|██████████| 19.4G/19.4G [06:34<00:00, 52.9MB/s]


In [4]:
import os
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
import torchvision.transforms as transforms

In [5]:
data_dir = './not-safe-for-work'

print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

['test', 'train']
['nsfw', 'sfw']


In [6]:
mean = torch.tensor([0.5, 0.5, 0.5], dtype=torch.float32)
std = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float32)

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomRotation(degrees=40),

    transforms.Resize(256),
    transforms.CenterCrop(256),

    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [7]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

In [8]:
trainset = ImageFolder(data_dir+'/train', transform=transform)
testset = ImageFolder(data_dir+'/test', transform=transform)

In [9]:
trainset

Dataset ImageFolder
    Number of datapoints: 98531
    Root location: ./not-safe-for-work/train
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.3)
               RandomRotation(degrees=[-40.0, 40.0], interpolation=nearest, expand=False, fill=0)
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(256, 256))
               ToTensor()
               Normalize(mean=tensor([0.5000, 0.5000, 0.5000]), std=tensor([0.2290, 0.2240, 0.2250]))
           )

In [29]:
trainset_subset = torch.utils.data.Subset(trainset, np.random.choice(len(trainset), 10000, replace=False))
testset_subset = torch.utils.data.Subset(testset, np.random.choice(len(testset), 1000, replace=False))

In [30]:
img, label = trainset_subset[0]
print(img.shape, label)
img

torch.Size([3, 256, 256]) 1


tensor([[[ 1.2929,  1.3443,  1.4470,  ..., -2.1834, -2.1834, -2.1834],
         [ 1.2929,  1.3100,  1.4470,  ..., -2.1834, -2.1834, -2.1834],
         [ 1.3957,  1.3443,  1.4128,  ..., -2.1834, -2.1834, -2.1834],
         ...,
         [-2.1834, -2.1834, -2.1834,  ...,  0.7107,  0.7278,  0.5566],
         [-2.1834, -2.1834, -2.1834,  ...,  0.3168,  0.6593,  1.0532],
         [-2.1834, -2.1834, -2.1834,  ...,  1.0703,  0.4024,  0.4024]],

        [[ 1.1642,  1.2167,  1.3393,  ..., -2.2321, -2.2321, -2.2321],
         [ 1.1817,  1.1992,  1.3393,  ..., -2.2321, -2.2321, -2.2321],
         [ 1.2868,  1.2342,  1.3043,  ..., -2.2321, -2.2321, -2.2321],
         ...,
         [-2.2321, -2.2321, -2.2321,  ...,  0.5690,  0.5865,  0.4114],
         [-2.2321, -2.2321, -2.2321,  ...,  0.1663,  0.5165,  0.9191],
         [-2.2321, -2.2321, -2.2321,  ...,  0.9366,  0.2539,  0.2539]],

        [[ 1.0370,  1.0719,  1.1416,  ..., -2.2222, -2.2222, -2.2222],
         [ 1.0022,  1.0196,  1.1416,  ..., -2

In [31]:
train_dataloader = torch.utils.data.DataLoader(
    dataset=trainset_subset,
    batch_size=16,
    shuffle=False,
    num_workers=2
)

In [32]:
test_dataloader = torch.utils.data.DataLoader(
    dataset=testset_subset,
    batch_size=16,
    shuffle=False,
    num_workers=2
)

Hidden Layers, Activation Functions and Non-Linearity

In [37]:
input_size = 65536*3
hidden_size = 500 
num_classes = 2
num_epochs = 10
batch_size = 100
learning_rate = 0.001 

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [39]:
import torch.nn as nn
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.TanH = nn.Tanh()
        self.l2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.l1(x)
        out = self.TanH(out)
        out = self.l2(out)
        # no activation and no softmax at the end
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [40]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate) 

In [42]:
n_total_steps = len(train_dataloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataloader):  
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        images = images.reshape(-1, 256*256*3).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
          print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/10], Step [10/625], Loss: 0.2664
Epoch [1/10], Step [20/625], Loss: 0.3763
Epoch [1/10], Step [30/625], Loss: 0.5690
Epoch [1/10], Step [40/625], Loss: 0.8374
Epoch [1/10], Step [50/625], Loss: 0.5718
Epoch [1/10], Step [60/625], Loss: 0.8701
Epoch [1/10], Step [70/625], Loss: 0.5113
Epoch [1/10], Step [80/625], Loss: 0.3658
Epoch [1/10], Step [90/625], Loss: 0.4565
Epoch [1/10], Step [100/625], Loss: 0.4613
Epoch [1/10], Step [110/625], Loss: 0.5829
Epoch [1/10], Step [120/625], Loss: 0.6000
Epoch [1/10], Step [130/625], Loss: 0.5048
Epoch [1/10], Step [140/625], Loss: 0.7548
Epoch [1/10], Step [150/625], Loss: 0.4604
Epoch [1/10], Step [160/625], Loss: 0.5904
Epoch [1/10], Step [170/625], Loss: 0.3119
Epoch [1/10], Step [180/625], Loss: 0.7902
Epoch [1/10], Step [190/625], Loss: 0.7821
Epoch [1/10], Step [200/625], Loss: 0.4121
Epoch [1/10], Step [210/625], Loss: 0.4916
Epoch [1/10], Step [220/625], Loss: 0.4490
Epoch [1/10], Step [230/625], Loss: 0.5961
Epoch [1/10], Step [

In [43]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_dataloader:
        images = images.reshape(-1, 256*256*3).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 1000 test images: {acc} %')

Accuracy of the network on the 1000 test images: 70.6 %
